In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows",40)
%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
    #kdd_train_2labels_y = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
    
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")
    #kdd_test_2labels_y = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")
    

In [3]:
from sklearn.preprocessing import LabelEncoder

le_2labels = LabelEncoder()
#dataset.y_train_2labels = le_2labels.fit_transform(dataset.kdd_train_2labels_y)
#dataset.y_test_2labels = le_2labels.transform(dataset.kdd_test_2labels_y)


In [4]:
from itertools import product
from sklearn.model_selection import train_test_split

class preprocessing:
    x_train = dataset.kdd_train_2labels.iloc[:,:-2].values
    y_train = dataset.kdd_train_2labels.iloc[:,-2:].values #np.array(dataset.y_train_2labels)

    x_test, y_test = (dataset.kdd_test_2labels.iloc[:,:-2].values, 
                      dataset.kdd_test_2labels.iloc[:,-2:].values #np.array(dataset.y_test_2labels)
                     )
    

In [5]:
from collections import namedtuple
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import regularizers

class Train:
    score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
    #model_detail = namedtuple("model_detail", ['epoch', 'no_of_features','hidden_layers', 'model'])
    scores = []
    predictions = {}
    #models = []
    def execute(x_train, x_test, 
                y_train, y_test, 
                input_dim, no_of_features, hidden_layers,
                epochs = 5, keep_prob = 0.9):
        
        print("Training for no_of_features: {}, hidden_layer: {}".format(no_of_features, hidden_layers
                                                                        ))
        model = Sequential()
        model.add(Dense(no_of_features, input_dim=input_dim, activation='relu'))
        model.add(Dropout(keep_prob))
        #model.add(BatchNormalization())
        
        for i in range(hidden_layers - 1):
            model.add(Dense(no_of_features, activation='relu'))
            model.add(Dropout(keep_prob))
            #model.add(BatchNormalization())

        
        model.add(Dense(2, activation='softmax'))
        
        optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-04, decay=0.1)
        model.compile(loss='mean_squared_error',
                      optimizer=optimizer,
                      metrics=['accuracy'])

        x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=.1)
        
        model.fit(x_train, y_train,
                  validation_data=(x_test, y_test),
                  epochs=epochs,
                  batch_size=128,
                  verbose = 1)
        
        curr_score_valid = model.evaluate(x_valid, y_valid) #, batch_size=128)
        curr_score_test = model.evaluate(x_test, y_test) #, batch_size=128)
        pred_value = model.predict(x_test)
        
        print("\n Train Accuracy: {}, Test Accuracy: {}".format(curr_score_valid[-1], curr_score_test[-1])  )
        
        Train.scores.append(Train.score(epochs,no_of_features,hidden_layers,curr_score_valid[1], curr_score_test[1]))
        #Train.models.append(Train.model_detail(epochs,no_of_features,hidden_layers,model))
        
        y_pred = pred_value
        #y_pred = pred_value[:,-1]
        #y_pred[y_pred >= pred_value[:,-1].mean()] = 1
        #y_pred[y_pred < pred_value[:,-1].mean()] = 0
        
        curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,-2], "Normal_prob":y_pred[:,-1]})
        Train.predictions.update({"{}_{}_{}".format(epochs,f,h):curr_pred})
                

Using TensorFlow backend.


In [6]:
#features_arr = [4, 8, 16, 32, 64, 128, 256, 1024]
#hidden_layers_arr = [2, 4, 6, 50, 100]

features_arr = [2, 4, 8, 16, 32]
hidden_layers_arr = [2, 6, 10]


for f, h in product(features_arr, hidden_layers_arr):
    Train.execute(preprocessing.x_train, preprocessing.x_test, 
                  preprocessing.y_train, preprocessing.y_test, 
                 122, f, h)

Training for no_of_features: 2, hidden_layer: 2
Train on 113375 samples, validate on 22544 samples
Epoch 1/5
113375/113375 [==============================] - 1s - loss: 0.2514 - acc: 0.5382 - val_loss: 0.3113 - val_acc: 0.6539
Epoch 2/5
113375/113375 [==============================] - 1s - loss: 0.2515 - acc: 0.5381 - val_loss: 0.3107 - val_acc: 0.6535
Epoch 3/5
113375/113375 [==============================] - 1s - loss: 0.2515 - acc: 0.5380 - val_loss: 0.3104 - val_acc: 0.6535
Epoch 4/5
113375/113375 [==============================] - 1s - loss: 0.2510 - acc: 0.5382 - val_loss: 0.3102 - val_acc: 0.6535
Epoch 5/5
21056/22544 [===========================>..] - ETA: 0s
 Train Accuracy: 0.6601841562058097, Test Accuracy: 0.6535220014194464
Training for no_of_features: 2, hidden_layer: 6
Train on 113375 samples, validate on 22544 samples
Epoch 1/5
113375/113375 [==============================] - 2s - loss: 0.2494 - acc: 0.5343 - val_loss: 0.5657 - val_acc: 0.4308
Epoch 2/5
113375/113375 [=

113375/113375 [==============================] - 1s - loss: 0.3250 - acc: 0.5246 - val_loss: 0.4307 - val_acc: 0.5692
Epoch 4/5
113375/113375 [==============================] - 1s - loss: 0.3257 - acc: 0.5238 - val_loss: 0.4307 - val_acc: 0.5692
Epoch 5/5
21216/22544 [===========================>..] - ETA: 0s
 Train Accuracy: 0.47007461504664455, Test Accuracy: 0.5692423704755145
Training for no_of_features: 16, hidden_layer: 6
Train on 113375 samples, validate on 22544 samples
Epoch 1/5
113375/113375 [==============================] - 3s - loss: 0.2588 - acc: 0.5314 - val_loss: 0.4180 - val_acc: 0.5669
Epoch 2/5
113375/113375 [==============================] - 2s - loss: 0.2583 - acc: 0.5321 - val_loss: 0.4180 - val_acc: 0.5667
Epoch 3/5
113375/113375 [==============================] - 2s - loss: 0.2581 - acc: 0.5328 - val_loss: 0.4180 - val_acc: 0.5665
Epoch 4/5
113375/113375 [==============================] - 2s - loss: 0.2579 - acc: 0.5324 - val_loss: 0.4180 - val_acc: 0.5663
Epoch

In [7]:
pd.DataFrame(Train.scores).sort('test_score', ascending=False)

/home/ritesh_malaiya/anaconda3/envs/p3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,epoch,no_of_features,hidden_layers,train_score,test_score
3,5,4,2,0.704159,0.773155
0,5,2,2,0.660184,0.653522
9,5,16,2,0.470075,0.569242
5,5,4,10,0.464359,0.568444
14,5,32,10,0.458247,0.568267
10,5,16,6,0.463089,0.566137
11,5,16,10,0.462216,0.562899
12,5,32,2,0.462931,0.511089
6,5,8,2,0.514050,0.491350
7,5,8,6,0.537943,0.454933


In [8]:
#for m in Train.models:
#    m.model.save("dataset/keras_model_epoch_{}_no_of_features_{}_hidden_layers_{}".format(m.epoch,
#                                                                                         m.no_of_features,
#                                                                                         m.hidden_layers))

In [9]:
pd.Panel(Train.predictions).to_pickle("dataset/keras_dense_nsl_kdd_predictions.pkl")
pd.DataFrame(Train.scores).to_pickle("dataset/keras_dense_nsl_kdd_scores.pkl")